In [6]:
from numba import cuda, float32
import numpy as np

@cuda.jit
def dot_kernel_numba(a, b, out, size):
    # ← At this point, a, b, out are ALREADY on the device (GPU)
    # This kernel executes on the GPU
    shared = cuda.shared.array(256, float32)
    
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    local_i = cuda.threadIdx.x
    
    if i < size:
        shared[local_i] = a[i] * b[i]
    else:
        shared[local_i] = 0.0
    
    cuda.syncthreads()
    
    stride = cuda.blockDim.x // 2
    while stride > 0:
        if local_i < stride:
            shared[local_i] += shared[local_i + stride]
        cuda.syncthreads()
        stride //= 2
    
    if local_i == 0:
        cuda.atomic.add(out, 0, shared[0])


def dot_cuda_numba(a, b):
    """Wrapper function - CPU code that manages device transfers"""
    
    # ========================================
    # HOST → DEVICE TRANSFERS (CPU → GPU)
    # ========================================
    
    # Transfer 'a' from CPU to GPU
    a_device = cuda.to_device(a.astype(np.float32))  # ← DEVICE TRANSFER 1
    
    # Transfer 'b' from CPU to GPU  
    b_device = cuda.to_device(b.astype(np.float32))  # ← DEVICE TRANSFER 2
    
    # Allocate 'out' directly on GPU (no transfer, just allocation)
    out = cuda.to_device(np.zeros(1, dtype=np.float32))  # ← DEVICE ALLOCATION
    
    # ========================================
    # KERNEL LAUNCH (runs on GPU)
    # ========================================
    
    size = a_device.shape[0]
    threads_per_block = 256
    blocks_per_grid = (size + threads_per_block - 1) // threads_per_block
    
    # Launch kernel - a_device, b_device, out are all on GPU
    dot_kernel_numba[blocks_per_grid, threads_per_block](
        a_device, b_device, out, size
    )
    
    # ========================================
    # DEVICE → HOST TRANSFER (GPU → CPU)
    # ========================================
    
    # Copy result back from GPU to CPU
    return out.copy_to_host()  # ← DEVICE TRANSFER 3


# Test
SIZE = 8
a = np.arange(SIZE, dtype=np.float32)
b = np.arange(SIZE, dtype=np.float32)

result = dot_cuda_numba(a, b)
expected = np.dot(a, b)

print(f"CUDA result: {result[0]:.6f}")
print(f"NumPy result: {expected:.6f}")
print(f"Match: {np.allclose(result[0], expected)}")

CUDA result: 140.000000
NumPy result: 140.000000
Match: True


/home/lelarge/miniconda3/envs/cs336-syst/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [2]:
import triton
import triton.language as tl
import torch

@triton.jit
def dot_kernel(
    a_ptr,      # Pointer to first input vector
    b_ptr,      # Pointer to second input vector  
    out_ptr,    # Pointer to output scalar
    size,       # Size of vectors
    BLOCK_SIZE: tl.constexpr,  # Elements per program
):
    # Program ID (analogous to blockIdx.x)
    pid = tl.program_id(0)
    
    # Compute offsets for this program's block
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    
    # Mask for boundary handling
    mask = offsets < size
    
    # Load data (coalesced, automatic)
    a = tl.load(a_ptr + offsets, mask=mask, other=0.0)
    b = tl.load(b_ptr + offsets, mask=mask, other=0.0)
    
    # Element-wise multiplication
    products = a * b
    
    # Reduce sum within this block (automatic parallel reduction!)
    block_sum = tl.sum(products)
    
    # Atomic add to output (single thread per block does this)
    tl.atomic_add(out_ptr, block_sum)


def dot_triton(a, b):
    """Wrapper function to launch the kernel"""
    # Allocate output
    out = torch.zeros(1, device=a.device, dtype=a.dtype)
    
    # Grid and block configuration
    size = a.shape[0]
    BLOCK_SIZE = 256  # Triton works well with larger blocks
    grid = lambda meta: (triton.cdiv(size, meta['BLOCK_SIZE']),)
    
    # Launch kernel
    dot_kernel[grid](a, b, out, size, BLOCK_SIZE=BLOCK_SIZE)
    
    return out


# Usage example
SIZE = 8
a = torch.arange(SIZE, dtype=torch.float32, device='cuda')
b = torch.arange(SIZE, dtype=torch.float32, device='cuda')

result_triton = dot_triton(a, b)
result_torch = torch.dot(a, b)

print(f"Triton result: {result_triton.item()}")
print(f"PyTorch result: {result_torch.item()}")
print(f"Match: {torch.allclose(result_triton, result_torch)}")

Triton result: 140.0
PyTorch result: 140.0
Match: True


In [9]:
from numba import cuda
import numpy as np

def show_gpu_info():
    if not cuda.is_available():
        print("No CUDA GPU available")
        return
    
    print(f"Number of GPUs: {len(cuda.gpus)}\n")
    
    for i, gpu in enumerate(cuda.gpus):
        print(f"{'='*60}")
        print(f"GPU {i}: {gpu.name.decode('utf-8')}")
        print(f"{'='*60}")
        
        with cuda.gpus[i]:
            device = cuda.get_current_device()
            
            # Helper function to safely get attribute
            def safe_get(attr_name, default="N/A"):
                try:
                    return getattr(device, attr_name)
                except AttributeError:
                    return default
            
            # Basic info
            print(f"\n📊 BASIC INFO:")
            print(f"  Compute Capability: {device.compute_capability}")
            print(f"  PCI Device ID: {device.id}")
            
            # Thread/Block limits
            print(f"\n🧵 THREAD LIMITS:")
            print(f"  Max threads per block: {safe_get('MAX_THREADS_PER_BLOCK')}")
            print(f"  Max block dimensions: {safe_get('MAX_BLOCK_DIM_X')} × {safe_get('MAX_BLOCK_DIM_Y')} × {safe_get('MAX_BLOCK_DIM_Z')}")
            print(f"  Max grid dimensions: {safe_get('MAX_GRID_DIM_X')} × {safe_get('MAX_GRID_DIM_Y')} × {safe_get('MAX_GRID_DIM_Z')}")
            print(f"  Warp size: {safe_get('WARP_SIZE')}")
            
            # Memory limits
            print(f"\n💾 MEMORY:")
            meminfo = cuda.current_context().get_memory_info()
            free_memory = meminfo[0]
            total_memory = meminfo[1]
            print(f"  Total global memory: {total_memory / 1024**3:.2f} GB")
            print(f"  Free memory: {free_memory / 1024**3:.2f} GB")
            print(f"  Used memory: {(total_memory - free_memory) / 1024**3:.2f} GB")
            
            shared_mem = safe_get('MAX_SHARED_MEMORY_PER_BLOCK')
            if shared_mem != "N/A":
                print(f"  Shared memory per block: {shared_mem / 1024:.2f} KB")
            
            shared_mem_sm = safe_get('MAX_SHARED_MEMORY_PER_MULTIPROCESSOR')
            if shared_mem_sm != "N/A":
                print(f"  Shared memory per SM: {shared_mem_sm / 1024:.2f} KB")
            
            const_mem = safe_get('TOTAL_CONSTANT_MEMORY')
            if const_mem != "N/A":
                print(f"  Constant memory: {const_mem / 1024:.2f} KB")
            
            # Multiprocessor info
            print(f"\n🔢 MULTIPROCESSORS:")
            mp_count = safe_get('MULTIPROCESSOR_COUNT')
            print(f"  Number of SMs: {mp_count}")
            
            max_threads_sm = safe_get('MAX_THREADS_PER_MULTIPROCESSOR')
            if max_threads_sm != "N/A":
                print(f"  Max threads per SM: {max_threads_sm}")
                warp_size = safe_get('WARP_SIZE', 32)
                if warp_size != "N/A":
                    print(f"  Max warps per SM: {max_threads_sm // warp_size}")
            
            max_blocks_sm = safe_get('MAX_BLOCKS_PER_MULTIPROCESSOR')
            if max_blocks_sm != "N/A":
                print(f"  Max blocks per SM: {max_blocks_sm}")
            
            # Register info
            print(f"\n📝 REGISTERS:")
            regs_block = safe_get('MAX_REGISTERS_PER_BLOCK')
            if regs_block != "N/A":
                print(f"  Registers per block: {regs_block}")
            
            regs_sm = safe_get('MAX_REGISTERS_PER_MULTIPROCESSOR')
            if regs_sm != "N/A":
                print(f"  Registers per SM: {regs_sm}")
            
            # Performance
            print(f"\n⚡ PERFORMANCE:")
            clock = safe_get('CLOCK_RATE')
            if clock != "N/A":
                print(f"  Clock rate: {clock / 1000:.2f} MHz")
            
            mem_clock = safe_get('MEMORY_CLOCK_RATE')
            if mem_clock != "N/A":
                print(f"  Memory clock rate: {mem_clock / 1000:.2f} MHz")
            
            mem_bus = safe_get('GLOBAL_MEMORY_BUS_WIDTH')
            if mem_bus != "N/A":
                print(f"  Memory bus width: {mem_bus} bits")
            
            l2_cache = safe_get('L2_CACHE_SIZE')
            if l2_cache != "N/A":
                print(f"  L2 cache size: {l2_cache / 1024:.2f} KB")
            
            # Features
            print(f"\n✨ FEATURES:")
            concurrent = safe_get('CONCURRENT_KERNELS')
            if concurrent != "N/A":
                print(f"  Concurrent kernels: {bool(concurrent)}")
            
            unified = safe_get('UNIFIED_ADDRESSING')
            if unified != "N/A":
                print(f"  Unified addressing: {bool(unified)}")
            
            ecc = safe_get('ECC_ENABLED')
            if ecc != "N/A":
                print(f"  ECC enabled: {bool(ecc)}")
            
            managed = safe_get('MANAGED_MEMORY')
            if managed != "N/A":
                print(f"  Managed memory: {bool(managed)}")
            
            # Calculate theoretical occupancy if we have the data
            if mp_count != "N/A" and max_threads_sm != "N/A":
                print(f"\n📈 THEORETICAL LIMITS:")
                max_blocks = max_threads_sm // 256
                print(f"  Max blocks per SM (with 256 threads/block): {max_blocks}")
                total_threads = mp_count * max_threads_sm
                print(f"  Theoretical max concurrent threads: {total_threads:,}")
            
        print()

show_gpu_info()

Number of GPUs: 1

GPU 0: NVIDIA H100 NVL

📊 BASIC INFO:
  Compute Capability: (9, 0)
  PCI Device ID: 0

🧵 THREAD LIMITS:
  Max threads per block: 1024
  Max block dimensions: 1024 × 1024 × 64
  Max grid dimensions: 2147483647 × 65535 × 65535
  Warp size: 32

💾 MEMORY:
  Total global memory: 93.12 GB
  Free memory: 91.82 GB
  Used memory: 1.30 GB
  Shared memory per block: 48.00 KB
  Shared memory per SM: 228.00 KB
  Constant memory: 64.00 KB

🔢 MULTIPROCESSORS:
  Number of SMs: 132

📝 REGISTERS:
  Registers per block: 65536
  Registers per SM: 65536

⚡ PERFORMANCE:
  Clock rate: 1785.00 MHz
  Memory clock rate: 2619.00 MHz
  Memory bus width: 6144 bits
  L2 cache size: 61440.00 KB

✨ FEATURES:
  Concurrent kernels: True
  Unified addressing: True
  ECC enabled: True
  Managed memory: True

